In [ ]:
#@title
# Import necessary libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
import pandas as pd
import numpy as np

---

<br>

## The Challenge

Your goal is to create a simulation that can determine the velocity and direction of a putt that will stop at the hole.  The initial location of the ball and the location of the hole should be system parameters for an individual simulation (and should be adjustable for a later simulation).

<br> 

The only predetermined parameter that you will need is the coefficient of rolling friction ($\mu = 0.15$).  If you choose to incorporate drag, you can assume that the coefficient of drag is $C_d = 0.7$, the radius of the ball is $r = 2.16 cm$, and the density of air is $1.3 kg/m^3$.

In [ ]:
#@title
from scipy.interpolate import interpn
from numpy import gradient
import matplotlib.pyplot as plt

# Upload elevation data for the green and put data in an array
filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/CT8_golf_green_data.xlsx'
#filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/CT8_golf_green_data_test.xlsx'
data = pd.read_excel(filename, header=0, index_col=0)
green = np.array(data)

# Calculate the gradients at the green so that they can
# be accessed using coordinate values
grad = gradient(green)
grad[0] = np.flip(np.rot90(grad[0]),1)
grad[1] = np.flip(np.rot90(grad[1]),1)
grad = np.flip(grad)

# Plot a topographic map of the green
fig, (ax_topo, ax_slope) = plt.subplots(nrows=1, ncols=2, figsize=(14,10))
cs = ax_topo.contour(np.array(data.columns), np.array(data.index), green, levels=15)
ax_topo.clabel(cs,colors='black', fmt = '%1.3f');
ax_topo.set(title='Elevation Map of Green', xlabel='X-coordinate', ylabel='Y-coordinate');

# Plot the slope of the green (arrows pointing downhill)
x, y = np.meshgrid(np.array(data.index), np.array(data.columns))
ax_slope.quiver(y,x,-grad[0],-grad[1]);
ax_slope.set(title='Slope Map of Green', xlabel='X-coordinate');

In [ ]:
def angle_to_components(mag,angle):
    theta = np.deg2rad(angle)
    x = mag * np.cos(theta)
    y = mag * np.sin(theta)
    return pd.Series(dict(x=x,y=y),dtype=float)

def make_system(params):
    #xi,yi,xh,yh,v_mag,angle,mu,C_d,rad,g,grad,points,t_end,b_ang,b_vel = params.values()
    xi, yi, xh, yh = params['xi'], params['yi'], params['xh'], params['yh']
    v_mag, angle, mu, C_d = params['v_mag'], params['angle'], params['mu'], params['C_d']
    rad, g, grad, points = params['rad'], params['g'], params['grad'], params['points']
    t_end, b_ang, b_vel = params['t_end'], params['b_ang'], params['b_vel']
    mass, rho = params['mass'], params['rho']

    # compute angle of straight line from initial position to hole
    deltax = xh - xi
    deltay = yh - yi
    angle_ih = (np.arctan2(deltay,deltax)/(np.pi))*180.0
    angle = angle_ih + angle
    
    # compute x and y components of velocity
    vx, vy = angle_to_components(v_mag,angle)
    
    # make the initial state
    init = pd.Series(dict(x=xi, y=yi, vx=vx, vy=vy))
    
    # compute the frontal area of the ball
    area = np.pi * (rad)**2

    return dict(init=init, xh=xh, yh=yh, angle=angle, mu=mu, C_d=C_d,
                rho=rho, area = area, g=g, grad=grad, mass = mass, 
                points=points, t_end=t_end)
    
def plot_trajectory(results,label,**options):
    x = results.x.values
    y = results.y.values
    x_vs_y = pd.Series(data=y,index=x)
    x_vs_y.plot(label=label,xlabel='x position (m)',
             ylabel='y position (m)',figsize=[6,9], xlim=[0,12.2],ylim=[0,21.3],**options)
    
def plot_trajectory1(results,system,**options):
    x = results.x.values
    y = results.y.values
    x_vs_y = pd.Series(data=y,index=x)
    y_arr = ([system['yh']])
    x_arr = ([system['xh']])
    hole_loc = pd.Series(data=y_arr,index=x_arr)

    fig, ax = plt.subplots(figsize=(14,10))
    cs = ax.contour(np.array(data.columns), np.array(data.index), green, levels=15)
    ax.clabel(cs,colors='black', fmt = '%1.3f');
    ax.set(title='Optimized Lag Putt', xlabel='X-coordinate', ylabel='Y-coordinate');

    x_vs_y.plot(xlabel='x position (m)',
             ylabel='y position (m)',figsize=[6,9], xlim=[0,12.2],ylim=[0,21.3],**options)
    hole_loc.plot(style = 'o')

In [ ]:
# Define the forces acting on the ball
def drag_force(V, system):
    rho, C_d, area = system['rho'], system['C_d'], system['area']
    
    # Find the magnitude and direction of the velocity
    vel_mag = np.sqrt(V.x**2 + V.y**2)
    if vel_mag != 0:
        dir = V/vel_mag
    else:
        dir = pd.Series(dict(x = 0, y = 0), dtype = float)

    # Find the magnitude of the drag force
    drag_mag = rho * vel_mag**2 * C_d * area * (1/2)

    # Define the direction of the force as opposite that of the  velocity
    # Notice that "dir" is a vector, so f_drag is vector too
    f_drag = drag_mag * -dir

    return f_drag

def fric_acc(x, y, V, slope_x, slope_y, mu):

     # Find the magnitude and direction of the velocity
    vel_mag = np.sqrt(V.x**2 + V.y**2)
    if vel_mag != 0:
        dir = V/vel_mag
    else:
        dir = pd.Series(dict(x = 0, y = 0), dtype = float)
    
    a_fric = -mu * g * dir
    
    # Find the frictional force in each direction
    #a_fric_x = -mu * g * np.cos(np.arctan2(slope_x,1.0))
    #a_fric_y = -mu * g * np.cos(np.arctan2(slope_y,1.0))

    return a_fric.x, a_fric.y

In [ ]:
def slope_func(t, state, system):
    x, y, vx, vy = state
    g, points, grad, mass = system['g'], system['points'], system['grad'], system['mass']
    
    # Find the slope at the current position in both directions
    slope_x = interpn(points, grad[0], [x, y], bounds_error=False)[0]
    slope_y = interpn(points, grad[1], [x, y], bounds_error=False)[0]

    # Find acceleration caused by drag
    V = pd.Series(dict(x=vx, y=vy),dtype=float)
    a_drag = drag_force(V, system) / mass

    # Find acceleration caused by friction
    a_fric_x, a_fric_y = fric_acc(x, y, V, slope_x, slope_y, mu)


    # Acceleration has to be defined as a vector too
    a_grav_x = np.sin(np.arctan2(slope_x,1.0))*(-g)
    a_grav_y = np.sin(np.arctan2(slope_y,1.0))*(-g) #slope_y/1.0*(-g) #
    
    A_x = a_fric_x + a_drag.x +  a_grav_x
    A_y = a_fric_y + a_drag.y  + a_grav_y
    #print(a_fric_y, a_drag.y, a_grav_y)

    # Stop motion once velocity is small enough or if near an edge

    if vx**2 + vy**2 < 0.001:
        A_x = 0.0
        A_y = 0.0
        V.x = 0.0
        V.y = 0.0


    return V.x, V.y, A_x, A_y

In [ ]:
# Stop the simulation if the ball goes off the green
def event_func(t, state, system):
    x, y, vx, vy = state
    near_edge_x_lo = x - points[0][0]
    near_edge_x_hi = points[0][-1] - x
    near_edge_y_lo = y - points[1][0]
    near_edge_y_hi = points[1][-1] - y
    near_edge = min(min(near_edge_x_lo, near_edge_x_hi), min(near_edge_y_lo, near_edge_y_hi))
    return near_edge - 0.01

In [ ]:
# Define a function that finds the best velocity for an angle
# This function creates the internal loop (optimizing velocity)
def dist_func_vel(v_mag,params):
    params_var = params.copy()
    params_var.update(v_mag=v_mag)
    system = make_system(params_var)
    results, details = run_solve_ivp(system, slope_func, events=event_func)
    if details.message == 'A termination event occurred.':
        total_dist = 100.0
    else:
        distance_from_hole_x = results.iloc[-1].x - system['xh']
        distance_from_hole_y = results.iloc[-1].y - system['yh']
        total_dist = (distance_from_hole_x**2 + distance_from_hole_y**2)**0.5
    # We want to minimize the final distance to the hole
    return total_dist

In [ ]:
# Define a function that finds the best angle for a given v_mag
# This function creates the external loop (optimizing angle)
def dist_func_ang(angle,params):
    params_var = params.copy()
    params_var.update(angle=angle)
    
    # For a given speed, find the angle that gives the maximum height at the wall
    best_vel = spo.minimize_scalar(dist_func_vel, args=params_var, options={'xtol': 0.1})
    params.update(b_ang=angle)
    params.update(b_vel = best_vel.x)
    print("Angle",angle,"Velocity magnitude", best_vel.x, "Distance to Hole", best_vel.fun)
    
    # We want to minimize the final distance to the hole
    return best_vel.fun


In [ ]:
# Run the simulation
def run_simulation(params):
    best_ang = spo.minimize_scalar(dist_func_ang,args=params, bounds=[-180,180], method='bounded', options={'xatol': 0.1})
    params.update(angle = params['b_ang'])
    params.update(v_mag = params['b_vel'])
    system = make_system(params)
    results, details = run_solve_ivp(system, slope_func, events=event_func)
    print("Final Angle",params['angle'],"Final velocity magnitude", params['b_vel'])
    plot_trajectory1(results,system)

In [ ]:
# Creating parameters and a system
# All values in base metric units

xi = 1.0; yi = 1.0     # Initial location of ball
xh = 9; yh = 15.0    # Location of the hole
v_mag = 12.0            # Initial velocity of ball
# Initial angle of launch, relative to straight path bt ball and hole
# A positive value is an angle to the left of the straight path
angle = -17.0      
mu = 0.15             # Coefficient of friction
C_d = 0.7               # Drag coefficient of the ball
rad = 0.0216           # Radius of the ball
mass = 0.046           # Mass of the ball
rho = 1.3              # Density of the air
g = 9.81               # Gravitational acceleration
t_end = 10.0           # Time to end of simulation
grad = grad            # Gradient map of the green
points = (np.array(data.columns), np.array(data.index))

params = dict(xi = xi, yi=yi, xh=xh, yh=yh, v_mag=v_mag, 
              angle=angle, mu=mu, C_d=C_d, rad=rad,mass=mass,rho=rho,
              g=g, grad=grad, points=points, t_end=t_end,
              b_ang=0.0, b_vel=0.0)

In [ ]:
run_simulation(params)

In [ ]:
system = make_system(params)
results, details = run_solve_ivp(system, slope_func, events=event_func)
print(details.message)
plot_trajectory1(results,system)
results.y.iloc[-1]